In [33]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from dotenv import load_dotenv
import asyncio
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

In [34]:
model_client = OpenAIChatCompletionClient(
    model = "gemini-2.5-flash",
    api_key = api_key,
)

In [35]:
add_agent_one = AssistantAgent(
    name="add_agent_one",
    description="An agent that just adds numbers.",
    model_client=model_client,
    system_message="Add 1 to the number and give result as output.if nothing is given then start adding from 0 or start with 0.",
)

add_agent_two = AssistantAgent(
    name="add_agent_two",
    description="An agent that just adds numbers.",
    model_client=model_client,
    system_message="Add 1 to the number which is given by the before agent from adding it from user input.",
)

add_agent_three = AssistantAgent(
    name="add_agent_three",
    description="An agent that just adds numbers.",
    model_client=model_client,
    system_message="Add 1 to the number which is given to you which is given by the before agent.",
)

### MaxMessageTermination

In [36]:
from autogen_agentchat.conditions import MaxMessageTermination

max_termination = MaxMessageTermination(2)

In [37]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    participants=[add_agent_one, add_agent_two, add_agent_three],
    termination_condition= max_termination,
)

In [38]:
from autogen_agentchat.ui import Console

await Console(team.run_stream(task = "the First Numer is 1"))

---------- TextMessage (user) ----------
the First Numer is 1
---------- TextMessage (add_agent_one) ----------
2


TaskResult(messages=[TextMessage(id='7e1b28e4-6d2b-48e2-a673-0b1008e307db', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 11, 7, 39, 53, 788294, tzinfo=datetime.timezone.utc), content='the First Numer is 1', type='TextMessage'), TextMessage(id='5998e1fa-829d-4630-89af-8b45f9e1fc3d', source='add_agent_one', models_usage=RequestUsage(prompt_tokens=36, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 7, 11, 7, 39, 54, 469916, tzinfo=datetime.timezone.utc), content='2', type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

### TextMentionTermination

In [39]:
plot_agent = AssistantAgent(
    name="PlotAgent",
    description="An agent that generates a plot based on user input.",    
    model_client=model_client,
    system_message="You are a creative plot writer. Generate imaginative plots in 15 - 20  words",
)

character_agent = AssistantAgent(
    name="CharacterAgent",
    description="An agent that generates characters based on user input.",
    model_client=model_client,
    system_message="You are a creative character designer. Develop the characters with sad backstores with 10 words.",
)

ending_agent = AssistantAgent(
    name="EndingAgent",
    description="An agent that generates an ending based on user input.",
    model_client=model_client,
    system_message="You are a creative ending writer Who specialises in writing the best ending of the story within 10 words and when complete make sure provide 'THE END' .",
)

In [40]:
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

text_termination = TextMentionTermination("THE END")

team = RoundRobinGroupChat(
    participants=[plot_agent, character_agent, ending_agent],
    termination_condition=text_termination,
)

In [41]:
await Console(team.run_stream(task ="A Love Story between a Girl and Boy."))

---------- TextMessage (user) ----------
A Love Story between a Girl and Boy.
---------- TextMessage (PlotAgent) ----------
They met in recurring dreams, unknowingly building a shared world. Reality united them years later, fulfilling a love whispered by destiny.
---------- TextMessage (CharacterAgent) ----------
**Character 1: The Girl**
**Name:** Elara
**Sad Backstory:** Her dreams were her only escape from a suffocating, bleak reality.

**Character 2: The Boy**
**Name:** Ronan
**Sad Backstory:** He woke alone, yearning for the unknown comfort of his dream world.
---------- TextMessage (EndingAgent) ----------
Reality embraced their dream, forever erasing lonely, yearning pasts.
THE END


TaskResult(messages=[TextMessage(id='54c3c2bd-1ecb-449d-a3a7-93640d72fa33', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 11, 7, 40, 12, 955076, tzinfo=datetime.timezone.utc), content='A Love Story between a Girl and Boy.', type='TextMessage'), TextMessage(id='1f51c91c-af53-4b3b-8d43-54a800c1a8fc', source='PlotAgent', models_usage=RequestUsage(prompt_tokens=31, completion_tokens=25), metadata={}, created_at=datetime.datetime(2025, 7, 11, 7, 40, 18, 556702, tzinfo=datetime.timezone.utc), content='They met in recurring dreams, unknowingly building a shared world. Reality united them years later, fulfilling a love whispered by destiny.', type='TextMessage'), TextMessage(id='9a5041ad-d21d-4a94-ba3a-b6c0a72dede7', source='CharacterAgent', models_usage=RequestUsage(prompt_tokens=57, completion_tokens=68), metadata={}, created_at=datetime.datetime(2025, 7, 11, 7, 40, 23, 438153, tzinfo=datetime.timezone.utc), content='**Character 1: The Girl**\n**Name:**

### Combined Termniation

In [42]:
combined_termination = TextMentionTermination("THE END") | MaxMessageTermination(5)

team = RoundRobinGroupChat(
    participants=[plot_agent, character_agent, ending_agent],
    termination_condition=combined_termination,
)

In [43]:
await Console(team.run_stream(task = "The Love Story between a Boy and Girl."))

---------- TextMessage (user) ----------
The Love Story between a Boy and Girl.
---------- TextMessage (PlotAgent) ----------
Childhood rivals, they only found common ground on a perilous adventure, discovering an unlikely, enduring love amidst the chaos.
---------- TextMessage (CharacterAgent) ----------
**Character 1: The Boy**
**Name:** Kael
**Sad Backstory:** His parents' relentless ambition stole his childhood and joy.

**Character 2: The Girl**
**Name:** Lyra
**Sad Backstory:** Feared for her sharp wit, she craved genuine connection.
---------- TextMessage (EndingAgent) ----------
Rivals united, finding love, joy, and true connection.
THE END


TaskResult(messages=[TextMessage(id='5542a88a-f339-489a-a0fc-695bfa37a03a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 11, 7, 48, 0, 648576, tzinfo=datetime.timezone.utc), content='The Love Story between a Boy and Girl.', type='TextMessage'), TextMessage(id='c9babed7-bdc0-42c5-afc5-053b1d420d58', source='PlotAgent', models_usage=RequestUsage(prompt_tokens=67, completion_tokens=24), metadata={}, created_at=datetime.datetime(2025, 7, 11, 7, 48, 5, 820263, tzinfo=datetime.timezone.utc), content='Childhood rivals, they only found common ground on a perilous adventure, discovering an unlikely, enduring love amidst the chaos.', type='TextMessage'), TextMessage(id='c5b2eba4-8332-4760-b8f3-d08044f359ef', source='CharacterAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=65), metadata={}, created_at=datetime.datetime(2025, 7, 11, 7, 48, 11, 345842, tzinfo=datetime.timezone.utc), content="**Character 1: The Boy**\n**Name:** Kael\n**S

### External Termination

In [44]:
plot_agent = AssistantAgent(
    name="PlotAgent",
    description="An agent that generates a plot based on user input.",    
    model_client=model_client,
    system_message="You are a creative plot writer. Generate imaginative plots in 15 - 20  words",
)

character_agent = AssistantAgent(
    name="CharacterAgent",
    description="An agent that generates characters based on user input.",
    model_client=model_client,
    system_message="You are a creative character designer. Develop the characters with sad backstores with 10 words.",
)

ending_agent = AssistantAgent(
    name="EndingAgent",
    description="An agent that generates an ending based on user input.",
    model_client=model_client,
    system_message="You are a creative ending writer Who specialises in writing the best ending of the story within 10 words and when complete make sure provide 'THE END' .",
)

In [ ]:
from autogen_agentchat.conditions import ExternalTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
import asyncio

external_termination = ExternalTermination()

team = RoundRobinGroupChat(
    [plot_agent, character_agent],
    termination_condition= external_termination
)

In [ ]:
run = await asyncio.create_task(Console(team.run_stream(task = "A Love Story between a boy and a girl.")))

await asyncio.sleep(0.2)
external_termination.set()

await run

---------- TextMessage (user) ----------
A Love Story between a boy and a girl.


---------- TextMessage (PlotAgent) ----------
He saw her soul, she healed his heart. Together, they navigated life's storms, their bond deepening into an unbreakable, forever love.
---------- TextMessage (CharacterAgent) ----------
**Character 1: Ethan**
Grief consumed him; silence was his only, lonely companion.

**Character 2: Luna**
Suffered silently, no one ever truly seeing her deepest wounds.
---------- TextMessage (EndingAgent) ----------
Healed, seen, loved; their souls finally found home.
THE END
---------- TextMessage (PlotAgent) ----------
Ethan's silent grief met Luna's unseen pain. Seeing each other, their love healed wounds, guiding their souls home.
---------- TextMessage (CharacterAgent) ----------
That's a profoundly beautiful and peaceful end for Ethan and Luna. "Healed, seen, loved; their souls finally found home" perfectly encapsulates the journey of two wounded individuals finding solace and belonging in each other. It's a very touching resolution.
